<a href="https://colab.research.google.com/github/takudzwa-coder/tools/blob/master/Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Module imports, do not modify these
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import cv2

In [4]:
# Comment the line below if you'd like to use the GPU; default device is CPU
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Don't modify these
IMAGE_HEIGHT = 60
IMAGE_WIDTH = 60
CHANNELS = 3
BATCH_SIZE = 8
EPOCHS = 30

In [ ]:
def getDataGenerators():
    """
    This function creates two data generators: a training data generator and a validation data generator.
    Both generators can be created by sub-setting an ImageDataGenerator object. The ImageDataGenerator object is
    responsible for (limited) image pre-processing, dataset splitting and augmentation.

    Task: create the image data generator and sub-set it into two generators: a training data generator and a
    validation data generator.

    Image data generator requirements:
        - normalise input images by dividing by 255.
        - set the amount validation data to be 10% of the training data

    Sub-set data generator requirements:
        - Use target image sizes as defined above
        - Use the batch size as defined above
        - Class mode must be 'categorical'
        - Color mode must be RGB
        - Enable dataset shuffling

    Hint: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

    Return: dataGenerator1, dataGenerator2 (You may use any variable name; this is just an example)
    """

    datasetDirPath = "./data/train/"

    # TODO: Set the ImageDataGeneratorObject
    ImageDataGeneratorObject = tf.keras.preprocessing.image_dataset_from_directory(
    datasetDirPath, #done
    labels="inferred", 
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=BATCH_SIZE, #done
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),#done
    shuffle=True,
)

    # TODO: Set the training data generator subset

    # TODO: Set the validation data generator subset

    return trainGenerator, validationGenerator



In [ ]:
def buildModel():
    """
    This function defines a tensorflow model to be used for training and testing. The model has to be defined and
    compiled with an associated optimizer and loss function.

    Tasks:
        - Define the input layer for the CNN model
        - Add a global average pooling layer to the model output
        - add a batch normalization layer
        - add a dense layer
            - 512 units
            - ReLU activation function
        - add a dense layer
            - 256 units
            - ReLU activation function
        - add a dense layer
            - 2 units
            - softmax activation function
        - Use SDG optimizer with a learning rate of 1e-5

    Hint: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer
    Hint: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers

    :return: compiled TF model
    """
    # TODO: Every model requires an input layer. Define the input layer using the above declared height,
    #  width and channels constants
    inputs = Input(shape=(None, None, None))
    
    # Store the EfficientNetB0 model in a variable. Check the parameters.
    baseModel = EfficientNetB0(include_top=False, weights=None, input_tensor=inputs, classes=2)
    baseModel.trainable = True
    
    # Set model output
    baseModelOutput = baseModel.output
    
    # TODO: add specified layers
    out = None
    
    # Define end-to-end model
    model = tf.keras.Model(inputs, out, name="EfficientNet")
    
    # TODO: Set SGD optimizer
    
    # Set metrics (feel free to add additional metrics)
    METRICS = [
        tf.keras.metrics.BinaryAccuracy(name='accuracy')
    ]
    
    # Compile the model and set the loss function.
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=METRICS)
    
    # Return model
    return model


In [ ]:

def trainer():
    """
    This function controls model training. Steps for this function include:
        - retrieving the dataset generators
        - building the TF model
        - training the model
        - saving the trained model
    :return: trained TF model
    """
    # Get the train and validation generators
    trainGen, valGen = getDataGenerators()
    # Get the model
    model = buildModel()
    # Fit the model to the data; use the .fit function
    model.fit(
        x=trainGen,
        steps_per_epoch=trainGen.samples // BATCH_SIZE,
        validation_data=valGen,
        validation_steps=valGen.samples // BATCH_SIZE,
        epochs=EPOCHS,
        verbose=1)

    model.save("ENmodel.h5")

    # Return trained model
    return model



In [ ]:

def getPredictions(image):
    model = tf.keras.models.load_model("ENmodel.h5")
    # print(model.summary())
    rgbImage = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    rgbImage = cv2.resize(rgbImage, (IMAGE_HEIGHT, IMAGE_WIDTH))
    rgbImage = rgbImage.astype(np.float64)
    rgbImage = np.expand_dims(rgbImage, axis=0)
    rgbImage = rgbImage / 255.
    predictions = model.predict(rgbImage)
    print(predictions)
    return predictions



In [ ]:

def modelTesting():
    datasetDirPath = "./data/test/"
    classes = ["live/", "spoof/"]
    model = tf.keras.models.load_model("ENmodel.h5")
    xTest = np.empty((400, IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS), dtype=np.float64)
    y = np.empty((400, 2), dtype=np.float64)

    classImageNames = os.listdir(datasetDirPath + classes[0])

    for i in range(200):
        image_1 = cv2.imread(datasetDirPath + classes[0] + classImageNames[i])
        image_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2RGB)
        image_1 = cv2.resize(image_1, (IMAGE_HEIGHT, IMAGE_WIDTH))
        image_1 = np.array(image_1)
        image_1 = image_1 / 255.
        xTest[i,] = image_1
        y[i] = [1.0, 0.0]

    classImageNames = os.listdir(datasetDirPath + classes[1])

    for i in range(200):
        image_1 = cv2.imread(datasetDirPath + classes[1] + classImageNames[i])
        image_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2RGB)
        image_1 = cv2.resize(image_1, (IMAGE_HEIGHT, IMAGE_WIDTH))
        image_1 = np.array(image_1)
        image_1 = image_1 / 255.
        xTest[200 + i,] = image_1
        y[200 + i] = [0.0, 1.0]

    # Evaluate the restored model
    loss, acc = model.evaluate(xTest, y, verbose=2)
    print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))


if __name__ == '__main__':
    # Model training
    model = trainer()
    # Model testing
    modelTesting()
